# Document's Features

#### · Display raw data from cluster

#### · Display raw data and located peaks recorded in cluster 

#### · Find peaks of recorded raw data with the most updated version of the algorithm 

In [7]:
import nirscloud_util_meta
import nirscloud_util_hdfs
import logging
import pandas as pd
import numpy as np
import plotly
import pyspark
import sys
from plotly.subplots import make_subplots
import math 
import plotly.graph_objects as go
import sys
import matplotlib.pyplot as plt
from numpy import matrix
from scipy.signal import find_peaks
from scipy import interpolate
import datetime
from scipy.signal import find_peaks,butter, lfilter, lfilter_zi, convolve,resample, correlate, iirnotch, filtfilt, stft
from scipy import signal


/usr/local/lib/python3.9/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [8]:
#!pip install pymongo==3.13.0

In [9]:
#import util

In [10]:
my_name = 'braulio.ramirez'

In [11]:
logger = logging.getLogger('jhub')
hostname = 'mongos.mongo.svc.cluster.local:27017'
pemkeyfile = '/etc/mongo/jhub-keypem.pem'
sslca = '/etc/mongo/root-ca.pem'

nirscloud_util_meta.init(logger, 'meta', hostname=hostname, ssl=True, cert=pemkeyfile, ca=sslca)

In [12]:
spark_kerberos_principal = my_name + '@BABYNIRS.ORG'

params = {
    'spark_kerberos_principal': spark_kerberos_principal,
}
nirscloud_util_hdfs.init('/etc/jhub/conf/production.ini', params=params)

2023-06-30 19:30:26 [INFO ] cfg#_init_hdfs_kinit@79: after hdfs_kinit: the_stdout: b'' the_stderr: b''
2023-06-30 19:30:26 [INFO ] client#__init__@192: Instantiated <KerberosClient(url='https://hdfs2.babynirs.org:9870;https://hdfs1.babynirs.org:9870;https://hdfs4.babynirs.org:9870')>.


### Pulling the recorded peaks from the cluster

In [13]:
hdfs_path = '_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=00/_patient_id=5984929'

In [14]:
kafka_topics = 'nk_rpeak2_NICU'
hdfs_prefix = '/kafka/topics/%s'% (kafka_topics)

In [15]:
def _get_df(hdfs_path, the_id):
    hdfs_path = hdfs_path
    full_path = nirscloud_util_hdfs.full_path(hdfs_prefix, hdfs_path)
    err, df = nirscloud_util_hdfs.from_hdfs_path(full_path)
    print(err, df)
    is_valid = df['id'] == the_id
    df = df[is_valid]
    return df

In [16]:
df_II_peaks = _get_df(hdfs_path, 'II')

2023-06-30 19:30:26 [INFO ] client#status@320: Fetching status for '/'.
2023-06-30 19:30:27 [INFO ] client#status@320: Fetching status for '/kafka/topics/nk_rpeak2_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=00/_patient_id=5984929'.
2023-06-30 19:30:27 [INFO ] client#status@320: Fetching status for '/'.
2023-06-30 19:30:27 [INFO ] client#status@320: Fetching status for '/kafka/topics/nk_rpeak2_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=00/_patient_id=5984929'.
2023-06-30 19:30:27 [INFO ] client#list@1123: Listing '/kafka/topics/nk_rpeak2_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=00/_patient_id=5984929'.
2023-06-30 19:30:27 [INFO ] client#list@1123: Listing '/kafka/topics/nk_rpeak2_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=00/_patient_id=5984929'.
2023-06-30 19:30:27 [INFO ] client#status@320: Fetching status for '/'.
2023-06-30 19:30:27 [INFO ] client#status@320: Fetching status for '/kafka/topics/

None       id             start_ns          rec_nano_ts       val      ver  \
0     II  1686372038314000000  1686372039283320820  1.149853  v0.0.23   
1     II  1686372038826000000  1686372039623391041  0.070611  v0.0.23   
2     II  1686372038826000000  1686372039963980267  0.471633  v0.0.23   
3     II  1686372039338000000  1686372040304866902  0.387760  v0.0.23   
4     II  1686372039850000000  1686372040645622712  0.322574  v0.0.23   
...   ..                  ...                  ...       ...      ...   
3476  II  1686373175978000000  1686373176977949007 -0.491993  v0.0.23   
3477  II  1686373176490000000  1686373177586859349  0.641429  v0.0.23   
3478  II  1686373177002000000  1686373178182592895  0.716261  v0.0.23   
3479  II  1686373177514000000  1686373178621707140  0.550078  v0.0.23   
3480  II  1686373178026000000  1686373179078254610  2.296289  v0.0.23   

     _device_id  _bed_id   _the_date _hr _patient_id  
0       Procyon  HA11-01  2023-06-10  00     5984929  
1       

In [17]:
df_II_peaks.sort_values(by=['rec_nano_ts'], inplace=True)
df_II_peaks.reset_index(drop=True, inplace=True)
df_II_peaks['shift_rec_nano_ts'] = df_II_peaks['rec_nano_ts'].shift(-1)
df_II_peaks['dif_rec_nano_ts'] = df_II_peaks['shift_rec_nano_ts'] - df_II_peaks['rec_nano_ts']
df_II_peaks;

In [18]:
values_p = np.asarray(df_II_peaks['val'].to_numpy())
raw_time_p = np.asarray(df_II_peaks['rec_nano_ts'].to_numpy(), dtype = 'int')
time_p = [np.datetime64(int(t),'ns') for t in raw_time_p]

## Pulling continuous signal data from cluster 

In [19]:
kafka_topics_cs = 'nk_waves_NICU'
hdfs_prefix_cs = '/nirscloud/agg_by_hr3/%s'% (kafka_topics_cs)

In [20]:
def _get_df_cs(hdfs_path, the_id):
    hdfs_path = hdfs_path + '/id=%s' % (the_id)
    full_path = nirscloud_util_hdfs.full_path(hdfs_prefix_cs, hdfs_path)
    err, df = nirscloud_util_hdfs.from_hdfs_path(full_path)
    return df

In [21]:
df_II_cs = _get_df_cs(hdfs_path, 'MDC_ECG_ELEC_POTL_II')
df_II_cs.sort_values(by=['_milli_ts'], inplace=True)
df_II_cs.reset_index(drop=True, inplace=True)
df_II_cs['shift_milli_ts'] = df_II_cs['_milli_ts'].shift(-1)
df_II_cs['diff_milli_ts'] = df_II_cs['shift_milli_ts'] - df_II_cs['_milli_ts']
df_II_cs;

2023-06-30 19:30:29 [INFO ] client#status@320: Fetching status for '/'.
2023-06-30 19:30:29 [INFO ] client#status@320: Fetching status for '/nirscloud/agg_by_hr3/nk_waves_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=00/_patient_id=5984929/id=MDC_ECG_ELEC_POTL_II'.
2023-06-30 19:30:29 [INFO ] client#status@320: Fetching status for '/'.
2023-06-30 19:30:29 [INFO ] client#status@320: Fetching status for '/nirscloud/agg_by_hr3/nk_waves_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=00/_patient_id=5984929/id=MDC_ECG_ELEC_POTL_II'.
2023-06-30 19:30:29 [INFO ] client#list@1123: Listing '/nirscloud/agg_by_hr3/nk_waves_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=00/_patient_id=5984929/id=MDC_ECG_ELEC_POTL_II'.
2023-06-30 19:30:29 [INFO ] client#status@320: Fetching status for '/nirscloud/agg_by_hr3/nk_waves_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=00/_patient_id=5984929/id=MDC_ECG_ELEC_POTL_II'.
2023-06-30 19:30:

In [22]:
values_cs = np.asarray(df_II_cs['val'].to_numpy())
raw_time_cs = np.asarray(df_II_cs['_milli_ts'].to_numpy(), dtype = 'int')
time_cs = [np.datetime64(int(t),'ms') for t in raw_time_cs]

In [23]:
time_pks = np.array([(pd.Timestamp(t).hour*60*60*1000) + (pd.Timestamp(t).minute*60*1000) + (pd.Timestamp(t).second*1000) + (pd.Timestamp(t).microsecond/1000) for t in time_p]);
time_cts = np.array([(pd.Timestamp(t).hour*60*60*1000) + (pd.Timestamp(t).minute*60*1000) + (pd.Timestamp(t).second*1000) + (pd.Timestamp(t).microsecond/1000) for t in time_cs]);
time_pks = time_pks - time_cts[0]; time_cts = time_cts - time_cts[0];

In [24]:
def time_f(time_array):

    mins = ((time_array/1000)/60).astype(int);
    sec = ((time_array/1000)%60).astype(int);
    mili = ((time_array - (mins*60*1000 + sec*1000))*100).astype(int)/100 #*100.astype(int)/100 is to round to two decimals

    return (np.array([str(minutes)+':'+str(seconds)+':'+str(milis) for minutes, seconds, milis in zip(mins,sec,mili)]))

In [25]:
class raw_sample_():
    
    def __init__(self,min_min,min_sec,max_min,max_sec,seconds_per_label):
        
        self.min_min = min_min
        self.min_sec = min_sec
        self.max_min = max_min
        self.max_sec = max_sec
        self.seconds_per_label = seconds_per_label

        self.min_time = int((self.min_min*60*1000)/4) + int(self.min_sec*1000/4)
        self.max_time = int((self.max_min*60*1000)/4) + int(self.max_sec*1000/4) + 1
        self.pks_time_min = (self.min_min*60*1000) + (self.min_sec*1000)
        self.pks_time_max = (self.max_min*60*1000) + (self.max_sec*1000)
        self.pks_time = time_pks[(self.pks_time_min > time_pks).argmin():(time_pks < self.pks_time_max).argmin()]
        self.pks_values = values_p[(self.pks_time_min > time_pks).argmin():(time_pks < self.pks_time_max).argmin()]
        
        self.time =time_cts[self.min_time:self.max_time]
        self.values = df_II_cs['val'].to_numpy()[self.min_time:self.max_time]

        self.fig = go.Figure()
        self.fig.add_trace(go.Scatter(line= dict(color='blue'),x=self.time, y=self.values,showlegend=False))
        self.fig.add_trace(go.Scatter(mode='markers', marker= dict(color='red'),x=self.pks_time, y=self.pks_values,showlegend=False))

        self.ticks_values = []
        x = self.time[0]
        while x < self.time[-1]:
            self.ticks_values = np.append(self.ticks_values,x)
            x = 1000*self.seconds_per_label + x

        self.time_format_axis = time_f(self.ticks_values)
        

### Functions

In [26]:
class sample_():
    
    def __init__(self,min_min,min_sec,max_min,max_sec,seconds_per_label,find_the_peaks,inverted):
        
        self.min_min = min_min
        self.min_sec = min_sec
        self.max_min = max_min
        self.max_sec = max_sec
        self.seconds_per_label = seconds_per_label

        self.min_time = int((self.min_min*60*1000)/4 + int(self.min_sec*1000)/4) 
        self.max_time = int((self.max_min*60*1000)/4 + int(self.max_sec*1000)/4) + 1

        
        time_zero = raw_time_cs[0]
        self.time = raw_time_cs[self.min_time:self.max_time] - time_zero
        if inverted:
            self.values = - df_II_cs['val'].to_numpy()[self.min_time:self.max_time]
        else: 
            self.values =  df_II_cs['val'].to_numpy()[self.min_time:self.max_time]

        self.fig = go.Figure()
        self.fig.add_trace(
                      go.Scatter(line= dict(color='blue'),x=self.time, y=self.values,showlegend=False))

        self.ticks_values = []
        x = self.time[0]
        while x < self.time[-1]:
            self.ticks_values = np.append(self.ticks_values,x)
            x = 1000*self.seconds_per_label + x

        self.time_format_axis = time_f(self.ticks_values)
        
        if find_the_peaks:
            self.peaks, _ = find_peaks(self.values, distance=100)
            self.fig.add_trace(
                      go.Scatter(mode='markers',x=self.time[self.peaks], y=self.values[self.peaks],showlegend=False))


In [27]:
#bs_r_window = baseline removal window;     #fs = sampling frequency                                      #trans_width = transition width of bandpass 
#numtaps = size of filter                   # mwi_windoow = moving window integration window



class sample_all_included():
    
    first_time = True
    
    def __init__(self,min_min,min_sec,max_min,max_sec,seconds_per_label,inverted, bs_r_window,band_start,band_stop,fs,trans_width,numtaps, 
                 mwi_window, test_segments,spline,res):
        
        self.min_min = min_min
        self.min_sec = min_sec
        self.max_min = max_min
        self.max_sec = max_sec
        self.seconds_per_label = seconds_per_label


        self.min_time = int((self.min_min*60*1000)/4 + int(self.min_sec*1000)/4) 
        self.max_time = int((self.max_min*60*1000)/4 + int(self.max_sec*1000)/4) + 1

                
        time_zero = raw_time_cs[0]
        self.time = raw_time_cs[self.min_time:self.max_time] - time_zero
        if inverted:
            self.values = - df_II_cs['val'].to_numpy()[self.min_time:self.max_time]
        else: 
            self.values =  df_II_cs['val'].to_numpy()[self.min_time:self.max_time]
        
       
        self.ticks_values = []
        x = self.time[0]
        while x < self.time[-1]:
            self.ticks_values = np.append(self.ticks_values,x)
            x = 1000*self.seconds_per_label + x

        self.time_format_axis = time_f(self.ticks_values)       
        
        #Filters' Parameters
        self.first_bs_r_window = bs_r_window
        self.first_band = [band_start, band_stop]
        self.first_mwi_window = mwi_window
        self.bs_r_window = bs_r_window
        self.trans_width = trans_width
        self.numtaps = numtaps
        self.fs = fs
        self.band = [band_start,band_stop]
        self.edges = [0, self.band[0] - trans_width, self.band[0], self.band[1], self.band[1] + self.trans_width, 0.5*fs]
        self.test_segments = test_segments
        self.segment_duration = 0.8
        self.spline = spline 
        self.res = res
        

        # Preprocessed Data    
        #self.bs_re, self.cut_pts = moving_average(self.bs_r_window, self.values)
        #self.prepro_data =  self.values[0:len(self.time)-self.cut_pts] - self.bs_re
        #self.prepro_x = self.time[0:len(self.time)-self.cut_pts]
        
        #Bandpass
        self.taps = signal.remez(self.numtaps, self.edges, [0, 1, 0], fs=self.fs)
        self.bp_x_in = np.append(self.prepro_data,np.zeros(len(self.taps)))
        self.bp_y = np.zeros(len(self.prepro_data)+len(self.taps))
        
        for j in range(0,len(self.prepro_data)+len(self.taps)): 
            sum = 0
            for k in range(0,len(self.taps)):
                sum = (self.bp_x_in[j-k]*self.taps[k]) + sum
            self.bp_y[j] = sum
            
        self.bp_y=np.delete(self.bp_y,[np.arange(0,int(len(self.taps)/2))])
        self.bp_x = self.time[0:len(self.bp_y)]

        
        #derivative
        self.der_y = np.zeros(len(self.bp_y))
        self.der_x = self.bp_x
        self.der_x_in = np.append(self.bp_y,np.zeros(2))

        for i in range(0,len(self.bp_y)):
            self.der_y[i] = (1/8) * (-(self.der_x_in[i-2]) - (2*self.der_x_in[i-1]) + (2*self.der_x_in[i+1]) + (self.der_x_in[i+2]))
        
        #squaring
        self.sq_y = (self.der_y) ** 2
        self.sq_x = self.der_x
        
        #Moving Window Integration
        self.mwi_window = mwi_window

        
        
    
    @property
    def bs_r_window(self): 
        return self._bs_r_window

    @bs_r_window.setter
    def bs_r_window(self,new_window_value):
        self._bs_r_window = new_window_value
        self.bs_re, self.cut_pts = moving_average(self._bs_r_window,self.values)
        self.prepro_data = self.values[0:len(self.time)-self.cut_pts] - self.bs_re
        self.prepro_x = self.time[0:len(self.time)-self.cut_pts]
        
        if (new_window_value != self.first_bs_r_window): 
            self.first_bs_r_window = 300
            self.bs_re, self.cut_pts = moving_average(self._bs_r_window,self.values)
            self.prepro_data = self.values[0:len(self.time)-self.cut_pts] - self.bs_re
            self.prepro_x = self.time[0:len(self.time)-self.cut_pts]
            
            self.edges = [0, self.band[0] - self.trans_width, self.band[0], self.band[1], self.band[1] + self.trans_width, 0.5*self.fs]
            self.taps = signal.remez(self.numtaps, self.edges, [0, 1, 0], fs=self.fs)
            self.bp_x_in = np.append(self.prepro_data,np.zeros(len(self.taps)))
            self.bp_y = np.zeros(len(self.prepro_data)+len(self.taps))
            
            for j in range(0,len(self.prepro_data)+len(self.taps)): 
                sum = 0
                for k in range(0,len(self.taps)):
                    sum = (self.bp_x_in[j-k]*self.taps[k]) + sum
                self.bp_y[j] = sum

            self.bp_y=np.delete(self.bp_y,[np.arange(0,int(len(self.taps)/2))])
            self.bp_x = self.time[0:len(self.bp_y)]
            
            #Derivative when bandpass is updated
            self.der_y = np.zeros(len(self.bp_y))
            self.der_x_in = np.append(self.bp_y,np.zeros(2))
            for i in range(0,len(self.bp_y)):
                self.der_y[i] = (1/8) * (-(self.der_x_in[i-2]) - (2*self.der_x_in[i-1]) + (2*self.der_x_in[i+1]) + (self.der_x_in[i+2]))

            self.der_x = self.bp_x

            #Squaring when bandpass is updatd
            self.sq_y = self.der_y ** 2
            self.sq_x = self.der_x

            #Movig window integgration when bandpass is updated 
            self.mwi_y = dif_eq_window_integration(self.sq_y,self.mwi_window,False)
            self.mwi_x = self.der_x

            #Fiducial points when bandpass is updated
            self.fiducial_points = find_fiducial_point(self.mwi_y,self.mwi_x,self.test_segments,self.segment_duration,self.mwi_window)
            self.max_x, self.max_y, self.peaks_x, self.peaks_y, self.interpolated_peak_x, self.interpolated_peak_y,self.max_ids = interpolation_spline(
                self.fiducial_points, self.time, self.values, self.spline, self.res)

            
        
    @property
    def band(self):
        return self._band
    
    @band.setter
    def band(self,new_band):
        self.band_start = new_band[0]; self.band_stop = new_band[1]
        self._band = [new_band[0],new_band[1]]
        if (new_band != self.first_band): 
            self.first_band = 0.01
            self.edges = [0, self._band[0] - self.trans_width, self._band[0], self._band[1], self._band[1] + self.trans_width, 0.5*self.fs]
            self.taps = signal.remez(self.numtaps, self.edges, [0, 1, 0], fs=self.fs)
            self.bp_x_in = np.append(self.prepro_data,np.zeros(len(self.taps)))
            self.bp_y = np.zeros(len(self.prepro_data)+len(self.taps))

            for j in range(0,len(self.prepro_data)+len(self.taps)): 
                sum = 0
                for k in range(0,len(self.taps)):
                    sum = (self.bp_x_in[j-k]*self.taps[k]) + sum
                self.bp_y[j] = sum

            self.bp_y=np.delete(self.bp_y,[np.arange(0,int(len(self.taps)/2))])
            self.bp_x = self.time[0:len(self.bp_y)]


            #Derivative when bandpass is updated
            self.der_y = np.zeros(len(self.bp_y))
            self.der_x_in = np.append(self.bp_y,np.zeros(2))
            for i in range(0,len(self.bp_y)):
                self.der_y[i] = (1/8) * (-(self.der_x_in[i-2]) - (2*self.der_x_in[i-1]) + (2*self.der_x_in[i+1]) + (self.der_x_in[i+2]))

            self.der_x = self.bp_x

            #Squaring when bandpass is updatd
            self.sq_y = self.der_y ** 2
            self.sq_x = self.der_x

            #Movig window integgration when bandpass is updated 
            self.mwi_y = dif_eq_window_integration(self.sq_y,self.mwi_window,False)
            self.mwi_x = self.der_x

            #Fiducial points when bandpass is updated
            self.fiducial_points = find_fiducial_point(self.mwi_y,self.mwi_x,self.test_segments,self.segment_duration,self.mwi_window)
            self.max_x, self.max_y, self.peaks_x, self.peaks_y, self.interpolated_peak_x, self.interpolated_peak_y,self.max_ids = interpolation_spline(
                self.fiducial_points, self.time, self.values, self.spline, self.res)
                    
    @property
    def mwi_window(self):
        return self._mwi_window
    
    @mwi_window.setter
    def mwi_window(self,new_window):
        self._mwi_window = new_window
        self.mwi_y = dif_eq_window_integration(self.sq_y,self.mwi_window,False)
        self.mwi_x = self.der_x
        self.fiducial_points = find_fiducial_point(self.mwi_y,self.mwi_x,self.test_segments,self.segment_duration,self.mwi_window)
        self.max_x, self.max_y, self.peaks_x, self.peaks_y, self.interpolated_peak_x, self.interpolated_peak_y,self.max_ids = interpolation_spline(
                self.fiducial_points, self.time, self.values, self.spline, self.res)
    
    

        
    
        #self.prepro_fig = go.Figure()
        #self.prepro_fig.add_trace(go.Scatter(line=dict(color='orange'),x = self.prepro_x, y = self.prepro_data),showlegend = show_legend)



In [28]:
# This function returns the indexes of the intaken data where the fiducial point is
def find_fiducial_point(y,x,test_segments,segment_duration,window_length):
    fiducial_point = []
    peak = 0
    found_p = False
    mwi_peak = []
    look_for_peaks = True
    counter = 0
    peak_average, first_peaks = first_peaks_height_av(y,test_segments,0.8)

    for i in range(1,len(x)-1):
        if look_for_peaks:    
            f_derivative = (y[i+1]-y[i])/(x[i+1]-x[i])

            if found_p:
                if f_derivative > 0: 
                    found_p = False
                    if (y[i]<(0.9)*peak) : 
                        found_p = True
                        
                if (peak/2>y[i]) :
                    fiducial_point = np.append(fiducial_point,i-window_length)
                    mwi_peak = np.append(mwi_peak,y[i])
                    found_p = False
                    peak = 0
                    look_for_peaks = False
                    if len(fiducial_point)>test_segments:
                        peak_average = np.average(mwi_peak)
                        
            else: 
                b_derivative = (y[i]-y[i-1])/(x[i]-x[i-1])
                if (f_derivative < 0) and (b_derivative > 0) and y[i]>0.5*peak_average: 
                    peak = y[i]
                    found_p = True
                    
        else: 
            counter = counter + 1
            if counter== 45:
                counter = 0 
                look_for_peaks = True
            

    
    fiducial_point = np.asarray(fiducial_point, dtype = 'int')

    return fiducial_point

In [29]:
def bandpass_filter(data, lowcut, highcut, signal_freq, filter_order):
    nyquist_freq = 0.5 * signal_freq
    low = lowcut / nyquist_freq
    high = highcut / nyquist_freq
    b, a = butter(filter_order, [low, high], btype="band")
    y = lfilter(b, a, data)
    return y

In [30]:
#found_peaks needs to be the index of the peaks in the raw_data_values or raw_data_time
# This function returns the maximum or the interpolated peaks and the interpolated peak itself
def interpolation(found_peaks,raw_data_time, raw_data_values):
    
    # First identify if we are past the peak or before 
    amount_of_peaks = len(found_peaks)
    corrected_peaks_dict = {}; interpolated_peak_y = {} ;interpolated_peak_x = {};
    peaks_x = []; peaks_y = [];

    f = np.zeros(3)

    for i in range (0,amount_of_peaks): 

        #print(i)

        index = found_peaks[i]
        point = raw_data_values[index]
        previous_point = raw_data_values[index-1]

        if previous_point < point: 
            while previous_point < point: 
                index = index + 1
                point = raw_data_values[index]
                previous_point = raw_data_values[index-1]

            # When the while isn't valid the maximum is located at index-1

            last_p = index + 2
            first_p = index - 3

        #point will be the point past the maximum 
        else: 
            while previous_point > point: 
                index = index - 1
                point = raw_data_values[index]
                previous_point = raw_data_values[index-1]

            #when the while finished or isn't valid the maximum is located at index
            last_p = index+3
            first_p = index-2

        # Were staying with the maximum, one forward, and two previous points 

        r = raw_data_values[[first_p,first_p+2,last_p-1,last_p]]
        x = raw_data_time[[first_p,first_p+2,last_p-1,last_p]]

        # interpolation 

        f[0] = (x[2] - x[0]) / (x[1] - x[0])
        f[1] = (x[3] - x[3]) / (x[1] - x[0])
        f[2] = (((x[3]**2) - (x[0]**2)) - (f[1]*((x[1]**2) - (x[0]**2)))) / (((x[2]**2) - (x[0]**2)) - (f[0]*((x[1]**2) - (x[0]**2))))


        d = ( r[3]-r[0] - ( f[1] * ( r[1] - r[0] ) ) - ( ( r[2] - r[0] - (f[0] * (r[1]-r[0]) ) )*f[1]) ) / ((x[3]**3) - (x[0]**3) - ( ( (x[1]**3) - (x[0]**3) ) * f[1]) - (f[2]*( ( (x[2]**3) - (x[0]**3) ) - (f[0] * ( (x[1]**3)-(x[0]**3) )  ) ) ))
        c = ( r[2] - r[0] - ( (r[1]-r[0])*f[0]) - ( d * ( (x[2]**3) - (x[0]**3) - (f[0]*( (x[1]**3) - (x[0]**3) ) ) ) ) ) / ( (x[2]**2) - (x[0]**2) - ( f[0] * ( (x[1]**2) - (x[0]**2) )  ) )
        b = ( r[1] - r[0] - (d*((x[1]**3) - (x[0]**3))) - (c*((x[1]**2) - (x[0]**2))) ) / (x[1]-x[0])
        a = r[0] - (b*x[0]) - (c*(x[0]**2)) - (d*(x[0]**3))

        x_interp = np.arange(x[0],x[2]+0.5,0.5)
        y = a + (b*x_interp) + (c*(x_interp**2)) + (d*(x_interp**3))
        max_index = np.where(max(y)==y)

        peaks_x = np.append(peaks_x,x_interp[max_index][0])
        peaks_y = np.append(peaks_y, y[max_index][0])
        interpolated_peak_x['peak: '+str(i)] = x_interp
        interpolated_peak_y['peak: '+str(i)] = y
        corrected_peaks_dict['peak: '+str(i) ] = [x_interp[max_index][0],y[max_index][0]]
        
        
    return peaks_x, peaks_y, interpolated_peak_x, interpolated_peak_y


In [31]:
matrices={}
three_spline_m = np.array([[4, 1, 0, 0, 0, 0]])
four_spline_m =  np.array([[4, 1, 0, 0, 0, 0, 0, 0]])

two_spline_m =  np.array([[4, 1, 0, 0], [1, 4, 1, 0], [0, 1 , 4 , 1], [0, 0, 1, 4]])

new_row_3 = np.array([1, 4, 1, 0, 0, 0,])
new_row_4 = np.array([1, 4, 1, 0, 0, 0, 0, 0])

for l in range(0,(6-2)):
    three_spline_m = np.vstack([three_spline_m, np.roll(new_row_3, l)])
three_spline_m = np.vstack([three_spline_m, np.array([[0, 0, 0, 0, 1, 4]])])

for u in range(0,(8-2)):
    four_spline_m = np.vstack([four_spline_m, np.roll(new_row_4, u)])
four_spline_m = np.vstack([four_spline_m, np.array([[0, 0, 0, 0, 0, 0, 1, 4]])])

inverse_two_spline_m = np.linalg.inv(two_spline_m); inverse_three_spline_m = np.linalg.inv(three_spline_m); inverse_four_spline_m = np.linalg.inv(four_spline_m)
matrices['2_splines'] = inverse_two_spline_m; matrices['3_splines'] = inverse_three_spline_m; matrices['4_splines'] = inverse_four_spline_m;

In [32]:
xn = np.arange(400,412 + 1000/5000 ,1000/5000) 

In [33]:
#found_peaks needs to be the index of the peaks in the raw_data_values or raw_data_time
# This function returns the maximum or the interpolated peaks and the interpolated peak itself
# splines is how many splines you want to carry out around the peak, minumum is 1 for it to have  
# one polynomial to the right and one to the left

def interpolation_spline(found_peaks, raw_data_time, raw_data_values, splines, resolution):
    
    res = 1000/resolution
    
    # First identify if we are past the peak or before 
    h = 4;
    max_indices = []
    
    #amount of control points
    amount_of_peaks = len(found_peaks)
    corrected_peaks_dict = {}; interpolated_peak_y = {} ;interpolated_peak_x = {};
    peaks_x = []; peaks_y = []; max_x = []; max_y = [];

    f = np.zeros(3)
    for i in range (0,amount_of_peaks): 

        #print(i)

        index = found_peaks[i]
        point = raw_data_values[index]
        previous_point = raw_data_values[index-1]

        #Finding the maximum
        
        if previous_point < point: 
            while previous_point < point: 
                index = index + 1
                point = raw_data_values[index]
                previous_point = raw_data_values[index-1]

            # When the while isn't valid the maximum is located at index-1
            max_idx = index - 1


        #point will be the point past the maximum 
        
        else: 
            while previous_point > point: 
                index = index - 1
                point = raw_data_values[index]
                previous_point = raw_data_values[index-1]

            #when the while finished or isn't valid the maximum is located at index
            
            max_idx = index
        
        max_indices = np.append(max_indices,max_idx)
        last_p = max_idx + splines + 2
        first_p = max_idx - (splines + 1)

        # r and x include from x_{-1} up to x_{n+3}
        r = raw_data_values[first_p:last_p+1]
        x = raw_data_time[first_p:last_p+1]

        # interpolation 
        
        
        #M_(-1)
        m_1 =  (r[2] - (2*r[1]) + r[0]) / (h**2)
        #M_(n+1)
        m_n_1 =  (r[-1] - (2*r[-2]) + r[-3]) /(h**2)
        ys =[]
        ys = np.append( ys, ( (6/(h**2)) * (r[0] - (2*r[1]) + r[2]) ) + m_1 )
        actual_cp = splines*2
        
        for n in range(1,(actual_cp-2)+1):
            ys = np.append( ys, (6/(h**2)) * (r[n] - (2*r[n+1]) + r[n+2]) )
        
        ys = np.append( ys, (6/(h**2)) * (r[actual_cp-1] - (2*r[actual_cp]) + r[actual_cp+1]) - m_n_1)
        
        
        # Now the define inverted matrices
        
        inverted_matrix = matrices[str(splines)+'_splines']
        
        
        Ms = np.matmul(inverted_matrix, ys)
        b = Ms/2
        #We are appending m_n_1 which is the variable for M_{n+1} because we need it to calculate the c's
        Ms = np.append(Ms,m_n_1)
        a = (Ms[1:actual_cp+1] - Ms[0:actual_cp])/(6*h)
        c = ((r[2:(actual_cp+1)+1] - r[1:actual_cp + 1])/h) - ((h/6)*(Ms[1:actual_cp+1]+(2*Ms[0:actual_cp])))
        d = r[1:actual_cp +1] 
        # Remember the plus one is to include the last element which is the actual_cp index other wise python doesn't touch the last elemment
        
        interpolated_y = []
        
        #Piecewise polynomials
        for k in range (1,(splines*2)): 
            x_int = np.arange(x[k],x[k+1],res)
            y_int = (a[k-1]*((x_int - x[k])**3)) + (b[k-1]*((x_int - x[k])**2)) + (c[k-1]*(x_int-x[k])) + d[k-1]
            interpolated_y = np.concatenate((interpolated_y,y_int))
            
        # We delete every last point so that it doesn't repeat with the first of the next piecewise polynomial
        # So this last block of code is to include the last interpolation with the last point
        l_cp = splines*2
        x_int = np.arange(x[l_cp],x[l_cp+1]+res,res)
        y_int = (a[l_cp-1]*((x_int - x[l_cp])**3)) + (b[l_cp-1]*((x_int - x[l_cp])**2)) + (c[l_cp-1]*(x_int-x[l_cp])) + d[l_cp-1]
        interpolated_y = np.concatenate((interpolated_y, y_int))
        interpolated_x = np.arange(x[1],x[(splines*2)+1]+res ,res)
        
        interpolated_peak_y['peak: '+str(i)] = interpolated_y; interpolated_peak_x['peak: '+str(i)] = interpolated_x; 
        
        # getting the maximum collecting  vallues
        max_index = np.where(max(interpolated_y)==interpolated_y)

        peaks_x = np.append(peaks_x,interpolated_x[max_index][0])
        peaks_y = np.append(peaks_y, interpolated_y[max_index][0])
        
        #Analytical Solution
        
        roots_1 = []; roots_2 = []; s_eval_roots = []; s2_eval_roots =[]
        aj = a[splines-1:splines+1]; bj = b[splines-1:splines+1]; cj = c[splines-1:splines+1]; dj = d[splines-1:splines+1]; xj = x[splines:splines+2]
        
        #Coefficients of general formula 1 is for the first piecewise poolynomial and 2 is for the second piecewise polynomial
        g_a1 = 3*aj[0]; g_b1 = (2*bj[0] - (2*xj[0]*3*aj[0])); g_c1 = (3*aj[0]*(xj[0]**2)) - (2*bj[0]*xj[0]) + cj[0]
        g_a2 = 3*aj[1]; g_b2 = (2*bj[1] - (2*xj[1]*3*aj[1])); g_c2 = (3*aj[1]*(xj[1]**2)) - (2*bj[1]*xj[1]) + cj[1]

        roots_1 = np.append(roots_1, (- g_b1 + np.sqrt((g_b1**2) - (4 * g_a1 * g_c1 )))/(2*g_a1) );
        roots_1 = np.append(roots_1, (- g_b1 - np.sqrt((g_b1**2) - (4 * g_a1 * g_c1 )))/(2*g_a1) );
        s_eval_roots = np.append(s_eval_roots, aj[0]*((roots_1[0]-xj[0])**3) + bj[0]*((roots_1[0]-xj[0])**2) + cj[0]*(roots_1[0]-xj[0]) + dj[0])
        s_eval_roots = np.append(s_eval_roots, aj[0]*((roots_1[1]-xj[0])**3) + bj[0]*((roots_1[1]-xj[0])**2) + cj[0]*(roots_1[1]-xj[0]) + dj[0])
        
        if roots_1[np.argmax(s_eval_roots)] <=xj[1]:
            max_x = np.append(max_x, roots_1[np.argmax(s_eval_roots)])
            max_y = np.append(max_y, s_eval_roots[np.argmax(s_eval_roots)])
            
        else: 
            roots_2 = np.append(roots_2, (- g_b2 + np.sqrt((g_b2**2) - (4 * g_a2 * g_c2 )))/(2*g_a2) );
            roots_2 = np.append(roots_2, (- g_b2 - np.sqrt((g_b2**2) - (4 * g_a2 * g_c2 )))/(2*g_a2) );
            s2_eval_roots = np.append(s2_eval_roots, aj[1]*((roots_2[0]-xj[1])**3) + bj[1]*((roots_2[0]-xj[1])**2) + cj[1]*(roots_2[0]-xj[1]) + dj[1])
            s2_eval_roots = np.append(s2_eval_roots, aj[1]*((roots_2[1]-xj[1])**3) + bj[1]*((roots_2[1]-xj[1])**2) + cj[1]*(roots_2[1]-xj[1]) + dj[1])
            max_x = np.append(max_x, roots_2[np.argmax(s2_eval_roots)])
            max_y = np.append(max_y, s2_eval_roots[np.argmax(s2_eval_roots)])
            
    max_indices = np.asarray(max_indices, dtype = 'int')
        
    #max_x and max_y is for analytical solutions, peaks_x and peaks_y is for maximum by collecting data, and interpolated_peak_x/y is the actual interpolation
    return max_x, max_y, peaks_x, peaks_y, interpolated_peak_x, interpolated_peak_y,max_indices


In [34]:
# This function takes two arrays of peaks, where each value in the array is the index of the peak, and finds 
# if any of them have missing peaks compared to the other array
# Index comparing is when you are comparing the index of two different arrays but you might want to compare actual elements

def find_missing_peaks(a_peaks, b_peaks, index_comparing):
    missing_b_peaks = []; missing_a_peaks = []

    if len(a_peaks)<len(b_peaks):
        #a_peaks is the shortest in length
        a_peaks=np.append(a_peaks,np.zeros(len(b_peaks) - len(a_peaks)))        

    else: 
        #b_peaks is the shortest in length
        b_peaks=np.append(b_peaks,np.zeros(len(a_peaks) - len(b_peaks)))

        
    difference = abs(a_peaks - b_peaks) #Subtraction order don't matter

    a_insertions = 0; b_insertions = 0;

    while (difference>70).any():
        

        #Here we first find in what index is where a mark is misssing

        index = np.min(np.where(difference>70))
        #This defines if either the short pivot or the long pivot array is missing a mark on a peak
        
        # If True then there's a missing peak in the b pivot

        if (a_peaks[index] - b_peaks[index]) < 0:
            

            #This is for handling the last zeros. When you arrive to the ending zeros it means that you will not do 
            # more insertions but rather replace the zeros for the peaks
            
            if a_peaks[index] == 0:
                a_peaks = np.insert(a_peaks,index,b_peaks[index])
                a_peaks = np.delete(a_peaks,-1)
                missing_a_peaks = np.append(missing_a_peaks,index-b_insertions)

            else:
                missing_b_peaks = np.append(missing_b_peaks,index-a_insertions)
                b_peaks = np.insert(b_peaks,index,a_peaks[index])
                b_insertions = b_insertions + 1
                
                
                # If last element is zero you can erease last element, otherwise append a 0 to the other array. 
                if b_peaks[-1] == 0 :
                    b_peaks = np.delete(b_peaks,-1)
                else:
                    a_peaks =  np.append(a_peaks,0)

        #There is a missing peak in the a_peaks
        else:
            
           #This is for handling the last zeros. 
            if b_peaks[index] == 0:

                b_peaks = np.insert(b_peaks,index,a_peaks[index])
                b_peaks = np.delete(b_peaks,-1)
                missing_b_peaks = np.append(missing_b_peaks,index-a_insertions)


            else:
                missing_a_peaks = np.append(missing_a_peaks,index - b_insertions)
                a_peaks = np.insert(a_peaks,index,b_peaks[index])
                a_insertions = a_insertions + 1
                
                if a_peaks[-1] == 0 :
                    a_peaks = np.delete(a_peaks,-1)
                else:
                    b_peaks =  np.append(b_peaks,0)
                
        difference = abs(a_peaks - b_peaks)
    
   
    if index_comparing: 
        missing_a_peaks = np.asarray(missing_a_peaks, dtype = 'int'); missing_b_peaks = np.asarray(missing_b_peaks, dtype = 'int')
    else: 
        missing_a_peaks = np.asarray(missing_a_peaks, dtype = 'float'); missing_b_peaks = np.asarray(missing_b_peaks, dtype = 'float')
        
        
    # The outcome are the indices that are missing in the other array. For example if a is missing the index N in the b array 
    # then the index N is stored in missing_a_peaks
    # missing_a_peaks are indices of the b array, with elements that a doesn't contain
    return missing_a_peaks, missing_b_peaks


In [35]:
# This moving average function which serves as a difference equation is a central moving average which means that you are 
# averaging a point taking into account both sides neighbors. So at last you will need to cut either (window_length-1)/2)
# or (window_length-2)/2) depending wether you are having an even or odd number in the window length

def moving_average(window_length, y):
    
    x=y
    M = window_length - 1
    z = np.zeros(M+1)
    x_modified = np.concatenate((z,x[0:len(x) - (M+1)]))
    y1= (1/(M+1)) * (x[0]-x_modified[0])
    yn = [y1]

    for n in range (1,len(x)):
        y_value = ((1/(M+1)) * (x[n]-x_modified[n])) + yn[n-1]
        yn = np.append(yn,y_value)

    if window_length% 2 == 1: 
        cutting_points = int((window_length-1)/2) 
        yn = yn[cutting_points:len(yn)]
        
    else: 
        cutting_points = int((window_length-2)/2)
        yn = yn[cutting_points:len(yn)]
        
    return yn, cutting_points

In [36]:
def average_filter(window_length,signal):

    m2 = window_length - 1

    u = np.heaviside(np.arange(0,len(signal)),1)
    u_sub = np.concatenate([np.zeros(window_length),np.heaviside(np.arange(0,len(signal)-(window_length)),1)])
    h_n_s= (1/(window_length)) * (u - u_sub)

    mov_av_s = np.convolve (h_n_s,signal)
    mov_av_s_sh = mov_av_s[int(m2/2):len(signal)+int(m2/2)]
    
    return mov_av_s_sh

In [37]:
def dif_eq_window_integration(function,window_length,divide):
    
    pivot_function = function
    
    for i in range(1,window_length):
        pivot_function = np.insert(pivot_function,0,0)
        pivot_function = np.delete(pivot_function,len(pivot_function)-1)
        #print(pivot_function)
        function = function + pivot_function

        if divide: 
            result = (1/N)*function
        else: 
            result = function
            
    return result

In [38]:
def first_peaks_height_av(y,segments,fraction_of_a_second):

    s = 0
    max_values = []
    
    f = 250*fraction_of_a_second

    while s < segments: 

        max_values = np.append(max_values,np.max(y[int(f*s):int(f*(s+1))]))
        s = s+1

    average_peak_h = np.sum(max_values)/segments
    
    return average_peak_h, max_values


## Cluster Sample

In [39]:
cluster_sample = raw_sample_(1,30,3,0,1)
fig_raw = cluster_sample.fig
fig_raw.update_layout(xaxis = dict(tickmode = 'array', tickvals = cluster_sample.ticks_values, ticktext = cluster_sample.time_format_axis),autosize=False,width=1400,
                  height=600,showlegend=False,title="Raw Data ", xaxis_title="Time", yaxis_title= 'Signal', 
                  font=dict(family="Avenir",size=14,color="Black"));

## Pan Tompkins

In [ ]:
# Sample One: sample_test

In [40]:
min_min_t = 1; min_sec_t = 30; max_min_t= 3; max_sec_t = 0; seconds_per_label_t = 1; inverted_t = False;   bs_r_window_t = 6; 
band_start_t = 3; band_stop_t = 100; fs_t= 250;  trans_width_t = 2.88; numtaps_t = 152; show_legend_t = False; mwi_window_t = 25 ;
test_segments_t =3;            spline_t = 4;       res_t = 5000;

sample_test = sample_all_included(min_min_t,min_sec_t,max_min_t,max_sec_t,seconds_per_label_t,inverted_t, bs_r_window_t,band_start_t,band_stop_t,fs_t,
                       trans_width_t,numtaps_t, mwi_window_t, test_segments_t, spline_t,res_t)

In [58]:
# Sample Two: sample_test_dif_wl    this sample will have different window length for the mean average subtraction

In [41]:
min_min_t = 1; min_sec_t = 30; max_min_t= 3; max_sec_t = 0; seconds_per_label_t = 1; inverted_t = False;   bs_r_window_t = 10; 
band_start_t = 3; band_stop_t = 100; fs_t= 250;  trans_width_t = 2.88; numtaps_t = 152; show_legend_t = False; mwi_window_t = 25 ;
test_segments_t =3;            spline_t = 4;       res_t = 5000;

sample_test_dif_wl = sample_all_included(min_min_t,min_sec_t,max_min_t,max_sec_t,seconds_per_label_t,inverted_t, bs_r_window_t,band_start_t,band_stop_t,fs_t,
                       trans_width_t,numtaps_t, mwi_window_t, test_segments_t, spline_t,res_t)

In [42]:
fig_raw = make_subplots(rows=3, cols=1, subplot_titles=("Peaks ", 'Mean Subtraction', 'Moving Window Integration'))

fig_raw.add_trace(go.Scatter(line = dict(color = 'blue'), x = sample_test.time,y = sample_test.values,showlegend=False),row=1, col=1)
fig_raw.add_trace(go.Scatter(mode = 'markers', x = sample_test.max_x,y = sample_test.max_y,showlegend=False),row=1, col=1)
fig_raw.add_trace(go.Scatter(x = sample_test.prepro_x,y = sample_test.prepro_data,showlegend=False),row=2,col=1)
fig_raw.add_trace(go.Scatter(line=dict(color='#0a4345'),x = sample_test.mwi_x,y = sample_test.mwi_y,showlegend=False),row=3,col=1)
fig_raw.add_trace(go.Scatter(marker=dict(color='red'),mode ='markers',x = sample_test.mwi_x[sample_test.fiducial_points],y = sample_test.mwi_y[sample_test.fiducial_points],
                              showlegend=False),row=3,col=1)



fig_raw.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample_test.ticks_values, ticktext = sample_test.time_format_axis),
                    xaxis2 = dict(tickmode = 'array', tickvals = sample_test.ticks_values, ticktext = sample_test.time_format_axis),
                    xaxis3 = dict(tickmode = 'array', tickvals = sample_test.ticks_values, ticktext = sample_test.time_format_axis))

fig_raw.update_layout(xaxis_range=[79000,85000], xaxis2_range=[79000,85000],xaxis3_range=[79000,85000],yaxis_range=[-1.5,1.3],
                       yaxis2_range=[-0.4,0.4], yaxis3_range=[-0.001,0.05])
fig_raw.update_layout(height=1200,width = 1200);

In [60]:
#fig_raw.show()

## Mising Peaks

In [44]:
missing_pt_p, missing_cluster_p = find_missing_peaks(sample_test.max_x, cluster_sample.pks_time, True)


In [45]:
fig_c_vs_s =go.Figure()

fig_c_vs_s.add_trace(go.Scatter(line= dict(color='blue'),x=cluster_sample.time, y=cluster_sample.values,name='Raw Data',showlegend=True))
    

fig_c_vs_s.add_trace(go.Scatter(mode='markers',marker=dict(size=12),x=cluster_sample.pks_time, y= cluster_sample.pks_values, name = 'Cluster', 
                           showlegend=True,))

fig_c_vs_s.add_trace(go.Scatter(mode='markers',marker=dict(size=10,color='#21B626'), x = sample_test.max_x, 
                          y = sample_test.max_y, name = 'Current Version', showlegend=True))

fig_c_vs_s.add_trace(go.Scatter(mode='markers',marker=dict(size=7,color='orange'), x = sample_test.max_x[missing_cluster_p], 
                           y= sample_test.max_y[missing_cluster_p], name = 'Missing Cluster Peaks', showlegend=True))

fig_c_vs_s.add_trace(go.Scatter(mode='markers',marker=dict(size=7,color='#00FCF5'),x = cluster_sample.pks_time[missing_pt_p] , y= cluster_sample.pks_values[missing_pt_p], 
                          name = 'Missing Current Version Peaks', showlegend=True))


fig_c_vs_s.update_layout(xaxis = dict(tickmode = 'array', tickvals = cluster_sample.ticks_values, ticktext = cluster_sample.time_format_axis),autosize=False,width=1200,height=630,
                    title='Missing Cluster Peaks: '+str(len(missing_cluster_p))+'           Missing Current Version Peaks: '+str(len(missing_pt_p)) + 
                   '            MAW: '+str(sample_test.bs_r_window) + '         Band: '+str(sample_test.band_start)+' - '+str(sample_test.band_stop));


In [63]:
#fig_c_vs_s.show()

##

## Comparing different window lengths against 6 window length in mean average subtraction

In [46]:
#Visually missing peaks

fig_ms_p = go.Figure()
missing_6_dict ={};  missing_dif_wl_dict={};
spline=4;
res = 5000
lengths = np.arange(7,20,1)


for N in lengths: 
    sample_test_dif_wl.bs_r_window = N
    missing_6_p, missing_dif_wl_p = find_missing_peaks(sample_test.max_x, sample_test_dif_wl.max_x, True)
    missing_6_dict['window_length_'+str(N)] = missing_6_p
    missing_dif_wl_dict['window_length_'+str(N)] = missing_dif_wl_p
    
    fig_ms_p.add_trace(go.Scatter(line= dict(color='blue'),x=sample_test.time, y=sample_test.values,name='Raw Data',showlegend=True, visible = False))
    

    fig_ms_p.add_trace(go.Scatter(mode='markers',marker=dict(size=12),x=sample_test.max_x, y= sample_test.max_y, name = '6 WL Peaks', 
                               showlegend=True, visible = False))
    fig_ms_p.add_trace(go.Scatter(mode='markers',marker=dict(size=10,color='#21B626'), x = sample_test_dif_wl.max_x, y = sample_test_dif_wl.max_y,
                               name = str(N)+' WL Peaks',showlegend=True, 
                              visible = False))
    fig_ms_p.add_trace(go.Scatter(mode='markers',marker=dict(size=7,color='orange'), x = sample_test_dif_wl.max_x[missing_6_p], 
                               y= sample_test_dif_wl.max_y[missing_6_p], name = '6 WL Missing Peaks', showlegend=True, visible = False))
    
    fig_ms_p.add_trace(go.Scatter(mode='markers',marker=dict(size=7,color='#00FCF5'),x = sample_test.max_x[missing_dif_wl_p] , y= sample_test.max_y[missing_dif_wl_p], 
                              name = str(N)+' WL Missing  Peaks', showlegend=True, visible = False))

fig_ms_p.data[5].visible = True

steps = []
start = 0
for i in range(0,len(lengths)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig_ms_p.data)},
              {"title": "Peaks identified with window length "+str(lengths[i])+ '   Amount of 6 WL Missing Peaks: '+
               str(len(missing_6_dict['window_length_'+str(lengths[i])])) +'    Amount of ' + str(lengths[i]) + ' WL Missing Peaks: '+ 
               str(len(missing_dif_wl_dict['window_length_'+str(lengths[i])]))}],  # layout attribute
    )
    step["args"][0]["visible"][start] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][start+1] = True
    step["args"][0]["visible"][start+2] = True
    step["args"][0]["visible"][start+3] = True
    step["args"][0]["visible"][start+4] = True
    
    start = start+5
    steps.append(step)
    
sliders = [dict(active=5, currentvalue={"prefix": "n: "}, pad={"t": 50}, steps=steps)]
fig_ms_p.update_layout(sliders=sliders);
fig_ms_p.update_layout(xaxis_range=[79000,85000]);
fig_ms_p.update_layout(xaxis = dict(tickmode = 'array', tickvals = cluster_sample.ticks_values, ticktext = cluster_sample.time_format_axis),autosize=False,width=1200,height=600); 

/tmp/ipykernel_16800/270883639.py:124: RuntimeWarning:

invalid value encountered in sqrt

/tmp/ipykernel_16800/270883639.py:125: RuntimeWarning:

invalid value encountered in sqrt



In [47]:
#fig_ms_p.show()

## Pre Processed data with different window lengths

In [50]:

fig_prepro = go.Figure()
missing_6_dict ={};  missing_dif_wl_dict={};
spline=4;
res = 5000
lengths = np.arange(7,20,1)


for N in lengths: 
    sample_test_dif_wl.bs_r_window = N
    
    fig_prepro.add_trace(go.Scatter(x = sample_test_dif_wl.prepro_x , 
                              y= sample_test_dif_wl.prepro_data, name = str(N)+' WL Prepro Data', showlegend=True, visible = False))

fig_prepro.data[3].visible = True

steps = []
start = 0
for i in range(0,len(lengths)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig_prepro.data)},
              {"title": 'Preprocessed data, window length: '+str(lengths[i])}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)
    
sliders = [dict(active=5, currentvalue={"prefix": "n: "}, pad={"t": 50}, steps=steps)]
fig_prepro.update_layout(sliders=sliders);
fig_prepro.update_layout(xaxis = dict(tickmode = 'array', tickvals = cluster_sample.ticks_values, ticktext = cluster_sample.time_format_axis),autosize=False,width=1200,height=600); 

/tmp/ipykernel_16800/270883639.py:124: RuntimeWarning:

invalid value encountered in sqrt

/tmp/ipykernel_16800/270883639.py:125: RuntimeWarning:

invalid value encountered in sqrt



In [51]:
#fig_prepro.show()

## Bandpass 

In [52]:

figbp = go.Figure()
missing_6_dict ={};  missing_dif_wl_dict={};
spline=4;
res = 5000
lengths = np.arange(7,20,1)


for N in lengths: 
    sample_test_dif_wl.bs_r_window = N
    
    figbp.add_trace(go.Scatter(x = sample_test_dif_wl.bp_x , 
                              y= sample_test_dif_wl.bp_y, name = str(N)+' WL in MS, Bandpass', showlegend=True, visible = False))

figbp.data[3].visible = True

steps = []
start = 0
for i in range(0,len(lengths)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(figbp.data)},
              {"title": 'Bandpass , window length in mean subtraction: '+str(lengths[i])}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)
    
sliders = [dict(active=5, currentvalue={"prefix": "n: "}, pad={"t": 50}, steps=steps)]
figbp.update_layout(sliders=sliders);
figbp.update_layout(xaxis = dict(tickmode = 'array', tickvals = cluster_sample.ticks_values, ticktext = cluster_sample.time_format_axis),autosize=False,width=1200,height=600); 

/tmp/ipykernel_16800/270883639.py:124: RuntimeWarning:

invalid value encountered in sqrt

/tmp/ipykernel_16800/270883639.py:125: RuntimeWarning:

invalid value encountered in sqrt



In [53]:
#figbp.show()

## Derivative

In [54]:

fig_der = go.Figure()
missing_6_dict ={};
spline=4;
res = 5000
lengths = np.arange(7,20,1)


for N in lengths: 
    sample_test_dif_wl.bs_r_window = N
    
    fig_der.add_trace(go.Scatter(x = sample_test_dif_wl.der_x , 
                              y= sample_test_dif_wl.der_y, name = str(N)+' WL in MS, Derivative', showlegend=True, visible = False))

fig_der.data[3].visible = True

steps = []
start = 0
for i in range(0,len(lengths)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig_der.data)},
              {"title": 'Derivative , window length in mean subtraction: '+str(lengths[i])}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)
    
sliders = [dict(active=5, currentvalue={"prefix": "n: "}, pad={"t": 50}, steps=steps)]
fig_der.update_layout(sliders=sliders);
fig_der.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample_test.ticks_values, ticktext = sample_test.time_format_axis),autosize=False,width=1200,height=600); 

/tmp/ipykernel_16800/270883639.py:124: RuntimeWarning:

invalid value encountered in sqrt

/tmp/ipykernel_16800/270883639.py:125: RuntimeWarning:

invalid value encountered in sqrt



In [55]:
#fig_der.show()

## Moving Window Integration + Fiducial Points

In [56]:
mwi_fp = go.Figure()
missing_6_dict ={};  missing_dif_wl_dict={};
spline=4;
res = 5000
lengths = np.arange(7,20,1)


for N in lengths: 
    sample_test_dif_wl.bs_r_window = N
    
    mwi_fp.add_trace(go.Scatter(x = sample_test_dif_wl.mwi_x , 
                              y= sample_test_dif_wl.mwi_y, name = str(N)+' Bandpass WL: '+str(N), showlegend=True, visible = False))
    mwi_fp.add_trace(go.Scatter(mode = 'markers', x = sample_test_dif_wl.mwi_x[sample_test_dif_wl.fiducial_points] , 
                              y= sample_test_dif_wl.mwi_y[sample_test_dif_wl.fiducial_points], name = str(N)+' Bandpass WL: '+str(N), 
                              showlegend=True, visible = False))

mwi_fp.data[3].visible = True

steps = []
start = 0
for i in range(0,len(lengths)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(mwi_fp.data)},
              {"title": 'Bandpass data, window length: '+str(lengths[i])}],  # layout attribute
    )
    step["args"][0]["visible"][start] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][start + 1] = True  # Toggle i'th trace to "visible"
    start = start + 2
    steps.append(step)
    
sliders = [dict(active=5, currentvalue={"prefix": "n: "}, pad={"t": 50}, steps=steps)]
mwi_fp.update_layout(sliders=sliders);
mwi_fp.update_layout(xaxis = dict(tickmode = 'array', tickvals = cluster_sample.ticks_values, ticktext = cluster_sample.time_format_axis),autosize=False,width=1200,height=600); 

/tmp/ipykernel_16800/270883639.py:124: RuntimeWarning:

invalid value encountered in sqrt

/tmp/ipykernel_16800/270883639.py:125: RuntimeWarning:

invalid value encountered in sqrt



In [57]:
#mwi_fp.show()

In [209]:
min_min_t = 1; min_sec_t = 30; max_min_t= 3; max_sec_t = 0; seconds_per_label_t = 1; inverted_t = False;   bs_r_window_t = 15; 
band_start_t = 3; band_stop_t = 100; fs_t= 250;  trans_width_t = 2.88; numtaps_t = 152; show_legend_t = False; mwi_window_t = 25 ;
test_segments_t =3;            spline_t = 4;       res_t = 5000;

recreated_cluster = sample_all_included(min_min_t,min_sec_t,max_min_t,max_sec_t,seconds_per_label_t,inverted_t, bs_r_window_t,band_start_t,band_stop_t,fs_t,
                       trans_width_t,numtaps_t, mwi_window_t, test_segments_t, spline_t,res_t)

### Recreating cluster sample 

In [277]:
comp_prepro = make_subplots(rows=2, cols=1, subplot_titles=("Cluster ", 'MWI: '+str(sample_test.bs_r_window)))
comp_prepro.add_trace(go.Scatter(x = recreated_cluster.prepro_x,y = recreated_cluster.prepro_data,showlegend=False),row=1, col=1)
comp_prepro.add_trace(go.Scatter(x = sample_test.prepro_x,y = sample_test.prepro_data,showlegend=False),row=2, col=1)

comp_prepro.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample_test.ticks_values, ticktext = sample_test.time_format_axis),
                          xaxis2 = dict(tickmode = 'array', tickvals = recreated_cluster.ticks_values, ticktext = recreated_cluster.time_format_axis))
comp_prepro.update_layout(yaxis_range=[-1.2,1.2],yaxis2_range=[-0.45,0.4])
comp_prepro.update_layout(height=900,width = 1200);

In [276]:
sample_test.band=[4,100]
missing_pt_p, missing_cluster_p = find_missing_peaks(sample_test.max_x, cluster_sample.pks_time, True)
comp_bandpass = make_subplots(rows=3, cols=1, subplot_titles=('Peaks','MWI: '+str(sample_test.bs_r_window) , 'Cluster'))
comp_bandpass.add_trace(go.Scatter(x=cluster_sample.time, y= cluster_sample.values, name = 'Cluster', showlegend=True,),row = 1, col=1)
comp_bandpass.add_trace(go.Scatter(mode='markers',marker=dict(size=12),x=cluster_sample.pks_time, y= cluster_sample.pks_values, name = 'Cluster', 
                           showlegend=True,),row = 1, col=1)

comp_bandpass.add_trace(go.Scatter(mode='markers',marker=dict(size=10,color='#21B626'), x = sample_test.max_x, 
                          y = sample_test.max_y, name = 'Current Version', showlegend=True),row = 1, col=1)

comp_bandpass.add_trace(go.Scatter(mode='markers',marker=dict(size=7,color='orange'), x = sample_test.max_x[missing_cluster_p], 
                           y= sample_test.max_y[missing_cluster_p], name = 'Missing Cluster Peaks', showlegend=True),row = 1, col=1)

comp_bandpass.add_trace(go.Scatter(mode='markers',marker=dict(size=7,color='#00FCF5'),x = cluster_sample.pks_time[missing_pt_p] , y= cluster_sample.pks_values[missing_pt_p], 
                          name = 'Missing Current Version Peaks', showlegend=True),row = 1, col=1)

comp_bandpass.add_trace(go.Scatter(x = sample_test.bp_x,y = sample_test.bp_y,showlegend=False),row=2, col=1)
comp_bandpass.add_trace(go.Scatter(x = recreated_cluster.bp_x,y = recreated_cluster.bp_y,showlegend=False),row=3, col=1)

comp_bandpass.update_layout(xaxis = dict(tickmode = 'array', tickvals = cluster_sample.ticks_values, ticktext = cluster_sample.time_format_axis),
                            xaxis2 = dict(tickmode = 'array', tickvals = sample_test.ticks_values, ticktext = sample_test.time_format_axis),
                            xaxis3 = dict(tickmode = 'array', tickvals = recreated_cluster.ticks_values, ticktext = recreated_cluster.time_format_axis))
comp_bandpass.update_layout(xaxis_range=[141300,142900],xaxis2_range=[141300,142900],yaxis_range=[-2.8,3],yaxis2_range=[-0.2,0.2])
comp_bandpass.update_layout(height=1200,width = 1200);